In [ ]:
!pip install requests
!pip install psycopg2
!pip install pandas

In [ ]:
!pip install scikit-learn

In [ ]:
import requests
import psycopg2
from psycopg2 import sql
import json
import math

url_api = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/inmueblesPorUsuario/get_filtro/'
url_api_inmuebles = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/inmuebles/'
url_api_login = 'https://arqui-sistema-recomendacion-85b7038cdf33.herokuapp.com/api/login/'

login_data = {
    'username': 'diego3026',
    'password': '3174748557d'
}

In [ ]:
def normalizacion(valor,valor_maximo,valor_minimo):
    valor_normalizado = (valor - valor_minimo) / (valor_maximo - valor_minimo) 
    return valor_normalizado

In [ ]:
class InmuebleUsuarioNormalizado():

    def __init__(self,inmueble,usuario,favorito,calificacion,clics,pesos):
        self.inmueble = inmueble
        self.usuario = usuario
        self.favorito = favorito
        self.calificacion = calificacion
        self.clics = clics
        self.pesos = pesos

    def get_clasificacion(self):
        clasificacion = self.pesos[0] * self.favorito + self.pesos[1] * self.calificacion + self.pesos[2] * self.clics
        return clasificacion


In [ ]:
def get_datos_api():
    response_login = requests.post(url=url_api_login, data=login_data)

    if response_login.status_code == 200:
        data = response_login.json()
    else:
        print(f'Error en la petición login: {response_login.status_code}')


    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url_api, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return 'Error en la petición: {response.status_code}'

In [ ]:
def consultar_base_de_datos(host, database, user, password, consulta, parametros=None):
    """
    Conecta a la base de datos PostgreSQL y ejecuta una consulta.

    :param host: Dirección del servidor PostgreSQL.
    :param database: Nombre de la base de datos.
    :param user: Nombre de usuario para la conexión.
    :param password: Contraseña del usuario.
    :param consulta: Consulta SQL a ejecutar.
    :param parametros: Parámetros para la consulta SQL (opcional).
    :return: Resultados de la consulta.
    """
    try:
        # Establecer la conexión a la base de datos
        conexion = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        cursor = conexion.cursor()

        # Ejecutar la consulta
        cursor.execute(consulta)

        nombres_columnas = [desc[0] for desc in cursor.description]

        # Obtener los resultados de la consulta
        resultados = cursor.fetchall()

        # Cerrar el cursor y la conexión
        cursor.close()
        conexion.close()

        return nombres_columnas, resultados
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'Error al acceder a la base de datos: {error}')
        return None

In [ ]:
host = 'inmuebles.postgres.database.azure.com'
database = 'postgres'
user = 'gidsyc'
password = 'Semillero2024'
consulta = 'SELECT id FROM inmobiliaria_inmueble'
consulta_user = 'SELECT id, username FROM inmobiliaria_usuario'

columnas, resultados = consultar_base_de_datos(host, database, user, password, consulta)

columnas_user, resultados_user = consultar_base_de_datos(host, database, user, password, consulta_user)

datosLimpios = {}

# if resultados:
#     print(columnas)
#     for fila in resultados:
#         print(fila[0])
# else:
#     print('No se obtuvieron resultados.')


# print("Usuarios")
# if resultados_user:
#     print(columnas_user)
#     for fila in resultados_user:
#         print(fila[1])
# else:
#     print('No se obtuvieron resultados.')


for item in resultados_user:
    usuario = item[1]
    if usuario not in datosLimpios:
        datosLimpios[usuario] = {}
        for inmueble in resultados:
            id_inmueble = inmueble[0]
            datosLimpios[usuario][id_inmueble] = None
    else:
        print(f"El inmueble '{id_inmueble}' ya existe")


In [ ]:
data = get_datos_api()

#favorito, calificacion, clics
pesos = [5, 4, 1]

calificaciones = []
clics = []
favoritos = []
for item in data:
    if item['calificacion'] != None:
        calificaciones.append(item['calificacion'])

    if item['numeroDeClicks'] != None:
        clics.append(item['numeroDeClicks'])

    if item['favorito'] != None:
        favoritos.append(item['favorito'])

promedio_calificaciones = 1
promedio_clics = 0
promedio_favoritos = 0
max_clics = 0

if len(calificaciones) > 0:
    promedio_calificaciones = sum(calificaciones) / len(calificaciones)

if len(clics) > 0:
    max_clics = max(clics)
    promedio_clics = sum(clics) / len(clics)

if len(favoritos) > 0:
    promedio_favoritos = sum(favoritos) / len(favoritos)

for value in data:
    id_inmueble = value['inmueble'] if value['inmueble']!=None else None
    usuario = value['usuario'] if value['usuario']!=None else None
    calificacion = normalizacion(value['calificacion'],5,1) if value['calificacion']!=None else promedio_calificaciones
    favorito = int(value['favorito']) if value['favorito']!=None else promedio_favoritos

    if max_clics > 0 and value['numeroDeClicks']!=None:
        clics = normalizacion(value['numeroDeClicks'],valor_maximo=max_clics,valor_minimo=0)
    else:
        clics = promedio_clics


    if id_inmueble is not None and usuario is not None:        
        inmueble_por_usuario = InmuebleUsuarioNormalizado(inmueble=id_inmueble,
                                                        usuario=usuario,
                                                        calificacion=calificacion,
                                                        favorito=favorito,
                                                        clics=clics,
                                                        pesos=pesos)
        datosLimpios[inmueble_por_usuario.usuario][inmueble_por_usuario.inmueble] = inmueble_por_usuario.get_clasificacion()

print(datosLimpios)


# Nombre del archivo donde se guardarán los datos
nombre_archivo = 'datosLimpios.json'

# Guardar el diccionario en un archivo JSON
with open(nombre_archivo, 'w') as archivo_json:
    json.dump(datosLimpios, archivo_json, indent=4)


In [ ]:
import pandas as pd

ratings_df = pd.DataFrame(datosLimpios).T

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(ratings_df.fillna(0))
cosine_sim_df = pd.DataFrame(cosine_sim, index=ratings_df.index, columns=ratings_df.index)
print("Matriz de similitud del coseno:\n", cosine_sim_df)

In [ ]:
def predict_ratings(user, ratings_df, sim_df):
    user_ratings = ratings_df.loc[user]  # Valoraciones del usuario objetivo
    sim_scores = sim_df[user]  # Similitudes del usuario con otros usuarios
    
    # Excluir el propio usuario de la suma ponderada
    sim_scores = sim_scores.drop(user)
    ratings_df = ratings_df.drop(user)
    
    # Suma ponderada de las valoraciones de los vecinos
    weighted_sum = (ratings_df.T * sim_scores).sum(axis=1)
    sim_sum = sim_scores.sum()
    
    predicted_ratings = weighted_sum / sim_sum
    predicted_ratings = predicted_ratings[user_ratings.isna()]
    predicted_ratings = predicted_ratings[predicted_ratings > 0]

    return predicted_ratings.sort_values(ascending=False)


predicted_ratings = predict_ratings("camilo", ratings_df, cosine_sim_df)
print("Predicciones de valoraciones para 'camilo':\n", predicted_ratings)

recommendations = predicted_ratings.index.tolist()
print(f"inmuebles recomendados para 'camilo': {recommendations}")


In [ ]:
def get_datos_api_inmuebles():
    response_login = requests.post(url=url_api_login, data=login_data)

    if response_login.status_code == 200:
        data = response_login.json()
    else:
        return f'Error en la petición login: {response_login.status_code}'


    token = data['access']
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url_api_inmuebles, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return 'Error en la petición: {response.status_code}'

In [ ]:
inmuebles = get_datos_api_inmuebles()

In [47]:
import re
def buscar_palabra_clave(texto, palabra_clave):
    patron = r'\b' + re.escape(palabra_clave) + r'\b'
    encontrado = re.search(patron, texto) is not None
    return texto, encontrado

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')

def extraer_todas_las_palabras(texto):
    # Tokenizar el texto
    texto = texto.replace('-',' ')
    palabras = word_tokenize(texto.lower())
    
    # Filtrar signos de puntuación
    palabras = [palabra for palabra in palabras if palabra not in string.punctuation]

    # Filtrar palabras vacías (stopwords)
    palabras = [palabra for palabra in palabras if palabra not in stopwords.words('spanish')]

    return palabras


In [59]:
inmueble_caracteristicas = []

for item in inmuebles:
    caracteristicas = item['caracteristicas']
    if (len(caracteristicas) > 0):
        for caracteristica in caracteristicas:
            nombre = caracteristica['nombre'].lower()
            dato = {'id_inmueble': item['id'], 'nombre_caracteristica': nombre}
            inmueble_caracteristicas.append(dato)

preferencia = 'comedor'

for item in inmueble_caracteristicas:
    texto, encontrada = buscar_palabra_clave(item['nombre_caracteristica'],preferencia.lower())
    if encontrada:
        print("----------------------")
        print("PALABRA ENCONTRADA")
        print("id del inmueble => "+str(item['id_inmueble']))
        print(texto)
    else:
        continue

----------------------
PALABRA ENCONTRADA
id del inmueble => 56
tipo-de-comedor-sala-comedor
----------------------
PALABRA ENCONTRADA
id del inmueble => 56
tipo-de-piso-en-comedor-otro
----------------------
PALABRA ENCONTRADA
id del inmueble => 58
comedor-auxiliar
----------------------
PALABRA ENCONTRADA
id del inmueble => 58
tipo-de-comedor-comedor-independiente
----------------------
PALABRA ENCONTRADA
id del inmueble => 55
tipo-de-comedor-comedor-independiente
----------------------
PALABRA ENCONTRADA
id del inmueble => 55
tipo-de-piso-en-comedor-baldosa
----------------------
PALABRA ENCONTRADA
id del inmueble => 111
comedor-auxiliar
----------------------
PALABRA ENCONTRADA
id del inmueble => 112
tipo-de-comedor-comedor-independiente
----------------------
PALABRA ENCONTRADA
id del inmueble => 113
tipo-de-comedor-sala-comedor
----------------------
PALABRA ENCONTRADA
id del inmueble => 114
tipo-de-comedor-sala-comedor
----------------------
PALABRA ENCONTRADA
id del inmueble =>